# Tanford’s Surface Solvation Revisited: Interactions in Caffeine-Salt Solutions
Stefan Hervø-Hansen,<sup>a,*</sup> Jakub Polák,<sup>b,*</sup>Markéta Tomandlová,<sup>b</sup> Joachim Dzubiella,<sup>c</sup>Jan Heyda,<sup>b,*</sup> and Mikael Lund.<sup>a,*</sup>

<sup>a</sup> Division of Theoretical Chemistry, Department of Chemistry, Lund University, Lund SE 221 00, Sweden. <br>
<sup>b</sup> Department of Physical Chemistry, University of Chemistry and Technology, Prague, Technická 5, CZ-16628 Praha 6, Czech Republic. <br>
<sup>c</sup> Physikalisches Institut, Albert-Ludwigs Universität Freiburg, Hermann-Herder-Straße 3, D-79104 Freiburg im Breisgau, Germany.<br><br>
<sup>*</sup>To whom correspondence may be addressed stefan.hervo_hansen@teokem.lu.se; jan.heyda@vscht.cz; mikael.lund@teokem.lu.se

## Introduction
We present a study about the interactions in caffeine-salt solutions using experimental and computational methods. To study highly concentrated systems feasibly, one commonly utilizes implicit solvation methods for the main solvent, which is usually water. Here we take it one step further and include implicit inclusion of salts and salt-specific effects. Inspired by Tanford’s surface solvation model, we construct a Solvent Accessible Surface Area (SASA) Hamiltonian to capture the effects of salt and solvent on the attraction and repulsion of solutes. The general applicability of the model comes via a solvent-salt-solute-dependent Transfer Free Energy (TFE), which modulates the strength of the attraction or repulsion between solute species. 


## Methods & Materials
Metropolis Monte Carlo simulations[<sup>1</sup>](#fn1) were conducted using the Faunus (2.4.2)[<sup>2</sup>](#fn2)[<sup>,3</sup>](#fn3). For the simulation of caffeine in the presence of implicit solvent and salt, our pseudo-SASA pair-potential was employed, which calculates the overlapping surface area between pairs of spherical particles. For the calculation of the excess pressure, the volume perturbation method[<sup>4</sup>](#fn4) was utilized, with isotropic scaling of the periodic box dimensions and centroid of the individual caffeine molecules. For the calculation of the difference in excess chemical potential for caffeine in water and salt solutions, the Widom insertion method[<sup>5</sup>](#fn5) was employed in the canonical ensemble and also obtained using the sampled density in the grand canonical ensemble.


## References
1. <span id="fn1"> N. Metropolis, A. W. Rosenbluth, M. N. Rosenbluth, A. H. Teller, E. Teller, Equation of State Calculations by Fast Computing Machines. The Journal of Chemical Physics 21, 1087–1092 (1953).</span><br>
2. <span id="fn2"> B. Stenqvist, A. Thuresson, A. Kurut, R. Vácha, M. Lund, Faunus– a flexible framework for Monte Carlo simulation. Molecular Simulation 39, 1233–1239 (2013).</span><br>
3. <span id="fn3"> M. Lund, M. Trulsson, B. Persson, Faunus: An object oriented framework for molecular simulation. Source Code Biol Med 3 (2008).</span><br>
4. <span id="fn4"> V. I. Harismiadis, J. Vorholz, A. Z. Panagiotopoulos, Efficient pressure estimation in molecular simulations without evaluating the virial. The Journal of Chemical Physics 105, 8469–8470 (1996).</span><br>
5. <span id="fn5"> B. Widom, Some Topics in the Theory of Fluids. The Journal of Chemical Physics 39, 2808–2812 (1963).</span><br>


## Import modules
_Run this cell only **once** to ensure `homedir` is correctly set!_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import jinja2 as jinja
import subprocess
import itertools
import pandas as pd
from IPython.display import Markdown
import scipy
import json

# Physical constants and unit conversion
kJmol_to_kT = 0.40338846309 # @298.15 Kelvin
litre_to_Åcube = 10**(27)   # Ångstrom cubed
RT = 0.00831446261*298.15   # kJ/mol
Na = 6.02214086*10**(23)    # 1/mol

plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

homedir = !pwd
homedir = homedir[0]
print(homedir)

## Overview of the Coarse-graining of Caffeine
<figure>
  <img src=Figures/CFF_CG.png alt="Caffeine coarse graining" style=width:40%>
  <center> <b> Figure 1</b>: Coarse-graining of caffeine (spheres) over the atomic representation of caffeine. The naming of the groups CF1-CF8 can be parameterized individually in the following or in the <code>templates/</code> directory. </center>
</figure>

## Osmotic Coefficient Calibration
The model calibration has been conducted by searching for a value for the surface tension ($\gamma$) following the coarse-graining scheme previously presented to reproduce the caffeine concentration dependency of the osmotic coefficient. The osmotic coefficient is defined as the ratio between the total pressure of a system and the pressure of an ideal gas at the same concentration and temperature. Mathematically one can write:
$$\varphi = \frac{p^{\text{id}}+p^{\text{ex}}}{p^{\text{id}}} \tag{eq. 1}$$
where $p^{\text{id}}$ is the ideal pressure and $p^{\text{ex}}$ is the excess pressure arrising from intermolecular interactions between molecules in the system. To introduce an excluded volume for the coarse grained caffeine atoms we utilize a Weeks-Chandler-Andersen (WCA) potential, while all further attraction and repulsion is introduced though the SASA pair potential. For simplicity the `tension` variable is limited to only affect CF7 and CF8, while CF1-CF6 have their `tension=0`.

### Template loading and simulation setups

In [ ]:
%cd -q $homedir
# Template prepration
templateLoader = jinja.FileSystemLoader(searchpath="Templates")
templateEnv = jinja.Environment(loader=templateLoader)
TEMPLATE_FILE = "osmotic_calibration_template.yml"
template = templateEnv.get_template(TEMPLATE_FILE)


# Simulation settings
caffeineConcentrations = np.arange(start=0.005, stop=0.113, step=0.005) # Caffeine concentrage range
equil_steps = 50000                                                     # Number of equilibration MC steps
prod_steps  = 2000000                                                   # number of protection MC steps (x10)
tension     = 0.0548                                                    # Atomic surface tension

### Conducting simulations
The simulations will be conducted by inserting 200 particles and adjusting the volume such that we obtain the desired concentration of caffeine.

In [ ]:
%cd -q $homedir
numOfSims = 0

# Input parameters for the simulation
jinjaInput = {
'Homedir': ''
'Length': '',
'Steps': '',
'Tension': '',
'SaltConc': '',
'EquilSteps': ''
}

for caffeineConcentration in Ccaffine_list:
    # Calculate box size
    boxLength = np.around(np.cbrt(200 * Na**(-1) / float(caffeineConcentration) * litre_to_Åcube))

    # Input parameters into templeate
    jinjaInput['Homedir'] = homedir
    jinjaInput['Length'] = boxLength
    jinjaInput['Steps'] = equil_steps + prod_steps
    jinjaInput['EquilSteps'] = equil_steps
    jinjaInput['Tension'] = tension
    jinjaInput['SaltConc'] = 0
    outputText = template.render(jinjaInput)

    # Write faunus input file
    filePath = "Simulations/osmoticCalibration/{0:.3g}/Caffeine_calibration.yml".format(caffeineConcentration)
    folderPath =  "Simulations/osmoticCalibration/{0:.3g}".format(caffeineConcentration)
    ymlFile = open(filePath, "w")
    ymlFile.write(outputText)
    ymlFile.close()
    !chmod u+x $filePath
    %cd -q $folderPath
    !./Caffeine_calibration.yml > Caffeine_calibration.json
    subprocess.call(['screen','-d','-m', 'nice', 'faunus', '-v6', '-i', 'Caffeine_calibration.json'])
    numOfSims += 1

print('Number of simulations submitted: {}'.format(numOfSims))

### Visualizing the calibration

In [ ]:
#### EXPERIMENTAL DATA ####
# https://doi.org/10.1007/BF00652586 (Žółkiewski)
conc_exp1 = [0.0096, 0.0197, 0.0219, 0.0285, 0.0298, 0.0327, 0.0425, 0.0439, 0.0499,
            0.0530, 0.0546, 0.0644, 0.0711, 0.0726, 0.0740, 0.0780, 0.0823, 0.0848, 0.0963]
osm_exp1 = [0.9375, 0.8122, 0.8219, 0.8070, 0.8054, 0.7951, 0.7294, 0.7403, 0.7214,         # (25.0 °C)
           0.7169, 0.7143, 0.6832, 0.6610, 0.6611, 0.6486, 0.6603, 0.6318, 0.6545, 0.6334]  # (25.0 °C)

# https://pubs.acs.org/doi/pdf/10.1021/j100544a026 (Cesàro et al.)
conc_exp2 = [0.1116, 0.0765, 0.0590, 0.0528, 0.0386, 0.0258] # (29.8 °C)
osm_exp2  = [0.622,  0.683,  0.723,  0.745,  0.781,  0.835]  # (29.8 °C)
conc_exp3 = [0.1027, 0.0895, 0.0702, 0.0509, 0.0296, 0.0203] # (35.0 °C)
osm_exp3  = [0.656,  0.680,  0.720,  0.755,  0.837,  0.877]  # (35.0 °C)
###########################

%cd -q $homedir
fig = plt.figure(figsize=(8.4,6))
ax = fig.add_subplot(111)

Clist = []              # Concentrations
Ocoefficient_list = []  # Mean of mean osmotic coefficients
Ocoefficient_error = [] # Standard deviations of mean osmotic coefficients

for caffeineConcentration in Ccaffine_list:
    with open ('Simulations/osmoticCalibration/{0:.3g}/out.json'.format(caffeineConcentration)) as json_file:
        data = json.load(json_file)
    pid = data['analysis'][0]['density']['molecular']['CFF1']['c/M'] * Na / litre_to_Åcube
        
    dV = 5              # Volume perturbation in cubic Ångstrom
    Boltzmann_factor = np.loadtxt("Simulations/osmoticCalibration/{0:.3g}/volume.dat".format(caffeineConcentration), unpack=True, usecols=(3), skiprows=1)
    Boltzmann_factor = np.reshape(Boltzmann_factor, (-1, 4)).mean(axis=0) # Averaging over 4 equal blocks.
    pex = np.log(Boltzmann_factor)/dV

    osmoticCoeffificent = (pid+pex)/pid
    
    Ocoefficient_list.append(osmoticCoeffificent.mean())
    Ocoefficient_error.append(osmoticCoeffificent.std())
    Clist.append(data['analysis'][0]['density']['molecular']['CFF1']['c/M'])

# Plot simulation data
ax.errorbar(Clist, Ocoefficient_list, yerr=Ocoefficient_error, linestyle='solid', linewidth=6,
            label='Coarse grained model', alpha=0.7, capsize=5)    
        
# plot experimental sets
ax.plot(conc_exp1, osm_exp1, 'ko', label='Žółkiewski', ms=12, alpha=0.7)
ax.plot(conc_exp2, osm_exp2, 'ko', label='Cesàro et al.', markerfacecolor='none', ms=12)

# Sets labels and limits
ax.set_xlabel(r'Caffeine concentration (mol/l)', weight='bold',  size=20)
ax.set_ylabel(r'Osmotic coefficient ϕ', weight='bold', size=20)

# Sets axes ticks
ax.minorticks_off()
ax.tick_params(which='both', width=2, direction='in')
ax.tick_params(which='major', length=7, direction='in')
ax.tick_params(which='minor', length=3, direction='in')
for tick in list(itertools.chain(*[ax.xaxis.get_major_ticks(), ax.yaxis.get_major_ticks()])):
    tick.label.set_fontsize(18)
    
# Legend
ax.legend(loc='best', fontsize=20, frameon=False)

fig.tight_layout()
fig.savefig('Figures/Osmotic_coefficient.pdf', bbox_inches='tight')

## Osmotic Coefficient Calibration: Behavour of the Parameter Space
It could be seen from the previous calibration that using `tension = 0.0548` yielded a too strong attraction at higher concentrations of caffeine. To make a more robust model at higher concentrations of caffeine and investigate the behavior of the parameter space on the curvature of the osmotic coefficient with the caffeine concentration we now introduce more parameters to our model. In specific we introduce the variables `tension_ring` and `tension_methyl`. `tension_ring` like the previous `tension` variable accounts for the surface tension on CF7 and CF8, while `tension_methyl` accounts for the surface tension on CF1, CF2, and CF3, being the methyl groups of caffeine.
### Template loading and simulation setups

In [ ]:
%cd -q $homedir
# Template prepration
templateLoader = jinja.FileSystemLoader(searchpath="Templates")
templateEnv = jinja.Environment(loader=templateLoader)
TEMPLATE_FILE = "osmotic_calibration_alternative_template.yml"
template = templateEnv.get_template(TEMPLATE_FILE)


# Simulation settings (shared parameters)
caffeineConcentrations = np.arange(start=0.005, stop=0.113, step=0.005) # Caffeine concentrage range
equil_steps = 50000                                                     # Number of equilibration MC steps
prod_steps  = 2000000                                                   # number of protection MC steps (x10)

# Model specific parameters
model1 = {'Name': 'Alternative_model_1', 'Tension_ring': 0.0480, 'Tension_methyl':  0.0100}
model2 = {'Name': 'Alternative_model_2', 'Tension_ring': 0.0660, 'Tension_methyl': -0.0200}
models = [model1, model2]

### Conduct simulations
The simulations will be conducted by inserting 200 particles and adjusting the volume such that we obtain the desired concentration of caffeine.

In [ ]:
%cd -q $homedir
numOfSims = 0

# Input parameters for the simulation
jinjaInput = {
'Homedir': ''
'Length': '',
'Steps': '',
'Tension_ring': '',
'Tension_methyl': '',
'SaltConc': '',
'EquilSteps': ''
}

for caffeineConcentration in Ccaffine_list:
    # Calculate box size
    boxLength = np.around(np.cbrt(200 * Na**(-1) / float(caffeineConcentration) * litre_to_Åcube))
    for model in models:
        # Input parameters into templeate
        jinjaInput['Homedir'] = homedir
        jinjaInput['Length'] = boxLength
        jinjaInput['Steps'] = equil_steps + prod_steps
        jinjaInput['EquilSteps'] = equil_steps
        jinjaInput['Tension_ring'] = model['Tension_ring']
        jinjaInput['Tension_methyl'] = model['Tension_methyl]
        jinjaInput['SaltConc'] = 0
        outputText = template.render(jinjaInput)

        # Write faunus input file
        filePath = "Simulations/osmoticCalibration_{0}/{1:.3g}/Caffeine_calibration.yml".format(model['name'], caffeineConcentration)
        folderPath =  "Simulations/osmoticCalibration_{0}/{1:.3g}".format(model['name'], caffeineConcentration)
        ymlFile = open(filePath, "w")
        ymlFile.write(outputText)
        ymlFile.close()
        !chmod u+x $filePath
        %cd -q $folderPath
        !./Caffeine_calibration.yml > Caffeine_calibration.json
        subprocess.call(['screen','-d','-m', 'nice', 'faunus', '-v6', '-i', 'Caffeine_calibration.json'])
        numOfSims += 1

print('Number of simulations submitted: {}'.format(numOfSims))

### Visualizing the alternative calibrations

In [ ]:
#### EXPERIMENTAL DATA ####
# https://doi.org/10.1007/BF00652586 (Žółkiewski)
conc_exp1 = [0.0096, 0.0197, 0.0219, 0.0285, 0.0298, 0.0327, 0.0425, 0.0439, 0.0499,
            0.0530, 0.0546, 0.0644, 0.0711, 0.0726, 0.0740, 0.0780, 0.0823, 0.0848, 0.0963]
osm_exp1 = [0.9375, 0.8122, 0.8219, 0.8070, 0.8054, 0.7951, 0.7294, 0.7403, 0.7214,         # (25.0 °C)
           0.7169, 0.7143, 0.6832, 0.6610, 0.6611, 0.6486, 0.6603, 0.6318, 0.6545, 0.6334]  # (25.0 °C)

# https://pubs.acs.org/doi/pdf/10.1021/j100544a026 (Cesàro et al.)
conc_exp2 = [0.1116, 0.0765, 0.0590, 0.0528, 0.0386, 0.0258] # (29.8 °C)
osm_exp2  = [0.622,  0.683,  0.723,  0.745,  0.781,  0.835]  # (29.8 °C)
conc_exp3 = [0.1027, 0.0895, 0.0702, 0.0509, 0.0296, 0.0203] # (35.0 °C)
osm_exp3  = [0.656,  0.680,  0.720,  0.755,  0.837,  0.877]  # (35.0 °C)
###########################

%cd -q $homedir
fig = plt.figure(figsize=(8.4,6))
ax = fig.add_subplot(111)

# Model specific parameters
model1 = {'Path profix': '', 'Label': 'Original model'}
model2 = {'Path profix': 'Alternative_model_1', 'Label': 'Alternative model'}
model3 = {'Path profix': 'Alternative_model_2', 'Label': 'Alternative model 2'}
models = [model1, model2, model3]

for model in models:
    Clist = []              # Concentrations
    Ocoefficient_list = []  # Mean of mean osmotic coefficients
    Ocoefficient_error = [] # Standard deviations of mean osmotic coefficients
    
    for caffeineConcentration in Ccaffine_list:
        with open ('Simulations/osmoticCalibration{0}/{1:.3g}/out.json'.format(model['Path profix'], caffeineConcentration)) as json_file:
            data = json.load(json_file)
        pid = data['analysis'][0]['density']['molecular']['CFF1']['c/M'] * Na / litre_to_Åcube
            
        dV = 5              # Volume perturbation in cubic Ångstrom
        Boltzmann_factor = np.loadtxt("Simulations/osmoticCalibration{0}/{1:.3g}/volume.dat".format(model['Path profix'],caffeineConcentration), unpack=True, usecols=(3), skiprows=1)
        Boltzmann_factor = np.reshape(Boltzmann_factor, (-1, 4)).mean(axis=0) # Averaging over 4 equal blocks.
        pex = np.log(Boltzmann_factor)/dV
    
        osmoticCoeffificent = (pid+pex)/pid
        
        Ocoefficient_list.append(osmoticCoeffificent.mean())
        Ocoefficient_error.append(osmoticCoeffificent.std())
        Clist.append(data['analysis'][0]['density']['molecular']['CFF1']['c/M'])
    
    # Plot simulation data
    ax.errorbar(Clist, Ocoefficient_list, yerr=Ocoefficient_error, linestyle='solid', linewidth=6,
                label=model['Label'], alpha=0.7, capsize=5)    
        
# plot experimental sets
ax.plot(conc_exp1, osm_exp1, 'ko', label='Žółkiewski', ms=12, alpha=0.7)
ax.plot(conc_exp2, osm_exp2, 'ko', label='Cesàro et al.', markerfacecolor='none', ms=12)

# Sets labels and limits
ax.set_xlabel(r'Caffeine concentration (mol/l)', weight='bold',  size=20)
ax.set_ylabel(r'Osmotic coefficient ϕ', weight='bold', size=20)

# Sets axes ticks
ax.minorticks_off()
ax.tick_params(which='both', width=2, direction='in')
ax.tick_params(which='major', length=7, direction='in')
ax.tick_params(which='minor', length=3, direction='in')
for tick in list(itertools.chain(*[ax.xaxis.get_major_ticks(), ax.yaxis.get_major_ticks()])):
    tick.label.set_fontsize(18)
    
# Legend
ax.legend(loc='best', fontsize=20, frameon=False)

fig.tight_layout()
fig.savefig('Figures/Osmotic_coefficient_alternatives.pdf', bbox_inches='tight')

## Transfer Free Energy Calibration to Model Salt-Specific Effects
To incorporate salt-specific effects into our model, we now have to search for transfer free energies (TFE, $\varepsilon_i$) values for the caffeine motifs, which can reproduce the change in excess chemical potential of caffeine going from pure water as a solvent to as salt solution with a finite concentration of salt ($\Delta \mu^{\mathrm{excess}}(C_{s})$). The excess chemical potential, at constant caffeine activity, is obtained by simulations in the grand canonical ensemble ($\mu VT$) as the logarithmic ratio between the chosen activity and the sampled mean density. Mathematically one may write
$$
\Delta \mu^{\mathrm{excess}}(C_{s}) = \mu^{\mathrm{excess}}(C_{s}) - \mu^{\mathrm{excess}}(0) = kT \left[  \ln\left(\frac{a}{\rho(C_{s})}\right) - \ln\left(\frac{a}{\rho(0)}\right) \right] = kT \ln \left(\frac{\rho(0)}{\rho(C_s)}\right) \tag{eq. 2}
$$
Thus the difference in chemical potential is simply given as the logarithmic ratio between the sampled densities for caffeine in pure water and electrolyte solution containing $C_{s}$ molar salt.

### Template loading and simulation setups
Based on the previous, our simulation strategy will be to fixate the caffeine activity and conduct simulations at various salt concentrations and TFE values.

In [ ]:
%cd -q $homedir
# Template prepration
templateLoader = jinja.FileSystemLoader(searchpath="Templates")
templateEnv = jinja.Environment(loader=templateLoader)
TEMPLATE_FILE = "TFE_calibration_template.yml"
template = templateEnv.get_template(TEMPLATE_FILE)

# Simulation settings
salt_concentration_list = [0.1, 0.2, 0.3, 0.4, 0.5]
caffine_activity = 0.026
tfe_list = [-0.0125, -0.008, -0.001, 0.003, 0.0178]
equil_steps = 50000                                                     # Number of equilibration MC steps
prod_steps  = 2000000                                                   # number of protection MC steps (x10)
box_length  = 150                                                       # Legnth of cubic box in Ångstrom

### Conduct simulations

In [ ]:
%cd -q $homedir
numOfSims = 0

# Input parameters for the simulation
jinjaInput = {
'Homedir': ''
'Length': '',
'Steps': '',
'CaffeineActivity': '',
'TFE': '',
'SaltConc': '',
'SelfEnergy': '',
'EquilSteps': ''
}

# Do a single calculation for pure water outside loop
# Input parameters into templeate
jinjaInput['Homedir'] = homedir
jinjaInput['Length'] = box_length
jinjaInput['Steps'] = equil_steps + prod_steps
jinjaInput['CaffeineActivity'] = caffine_activity
jinjaInput['TFE'] = 0
jinjaInput['SaltConc'] = 0
jinjaInput['SelfEnergy'] = 0
outputText = template.render(jinjaInput)

# Write faunus input file
filePath = "Simulations/TFECalibration/{}_{}/TFE_calibration.yml".format(salt_concentration, tfe)
folderPath =  "Simulations/TFECalibration/{}_{}/".format(salt_concentration, tfe)
ymlFile = open(filePath, "w")
ymlFile.write(outputText)
ymlFile.close()
!chmod u+x $filePath
%cd -q $folderPath
!./TFE_calibration.yml > TFE_calibration.json
subprocess.call(['screen','-d','-m', 'nice', 'faunus', '-v6', '-i', 'TFE_calibration.json'])
numOfSims += 1


for salt_concentration in salt_concentration_list:
    for tfe in tfe_list:
        # Input parameters into templeate
        jinjaInput['Homedir'] = homedir
        jinjaInput['Length'] = box_length
        jinjaInput['Steps'] = equil_steps + prod_steps
        jinjaInput['CaffeineActivity'] = caffine_activity
        jinjaInput['TFE'] = tfe
        jinjaInput['SaltConc'] = salt_concentration
        jinjaInput['SelfEnergy'] = ((salt_concentration*tfe*4*np.pi*(4.64/2 + 1.3)**2 * 0.40338846309)+ # CF7
                                    (salt_concentration*tfe*4*np.pi*(4.18/2 + 1.3)**2 * 0.40338846309)) # CF8
        outputText = template.render(jinjaInput)

        # Write faunus input file
        filePath = "Simulations/TFECalibration/{}_{}/TFE_calibration.yml".format(salt_concentration, tfe)
        folderPath =  "Simulations/TFECalibration/{}_{}/".format(salt_concentration, tfe)
        ymlFile = open(filePath, "w")
        ymlFile.write(outputText)
        ymlFile.close()
        !chmod u+x $filePath
        %cd -q $folderPath
        !./TFE_calibration.yml > TFE_calibration.json
        subprocess.call(['screen','-d','-m', 'nice', 'faunus', '-v6', '-i', 'TFE_calibration.json'])
        numOfSims += 1

print('Number of simulations submitted: {}'.format(numOfSims))

### Visualizing the TFE calibrations

In [ ]:
%cd -q $homedir

# Experimental data
exp_data = pd.read_csv('chem_pot_experimental.csv', header=None, skiprows=2)
exp_data.loc[-1] = [0 for i in range(len(exp_data.columns))] # adding row of 0s
exp_data.index = exp_data.index + 1 # shifting index
exp_data = exp_data.sort_index() # sorting by index

# Simulation settings
salt_concentration_list = [0.1, 0.2, 0.3, 0.4, 0.5]
tfe_list = [-0.0125, -0.008, -0.001, 0.003, 0.0178]

fig, ax = plt.subplots(figsize=(8,5))

labels = ['Na₂SO₄', 'Sucrose' ,'NaCl', 'NaBr', 'NaSCN', 'NaClO₄']
def fitting_func(x, a):
    return a * x

j = 0
for i in range(len(exp_data.columns)):
    if i == 2:
        continue
    if (i % 2) == 0: # a even number check
        ax.plot(exp_data[i], exp_data[i+1], alpha=1.0, color='C{}'.format(j), marker='o', markerfacecolor='none', linestyle='none', markersize=8)
              
        popt, pcov = scipy.optimize.curve_fit(fitting_func, exp_data[i].dropna(), exp_data[i+1].dropna())
        
        # Calculating R**2 value:
        residuals = exp_data[i+1].dropna() - fitting_func(exp_data[i].dropna(), popt)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((exp_data[i+1].dropna()-np.mean(exp_data[i+1].dropna()))**2)
        r_squared = 1 - (ss_res / ss_tot)
        print('R² value for {cosolute}: \t{rvalue}'.format(cosolute=labels[int(i/2)], rvalue=r_squared))
        
        x = np.arange(0, 1.0, 0.0001)
        y = popt * x
        ax.plot(x, y, color='C{}'.format(j), linestyle='solid', marker='None', alpha=0.50, lw=5)
        
        j += 1
    else:
        continue


# Caffeine in water
%cd -q $homedir
with open ('Simulations/TFECalibration/0_0/out.json') as json_file:
    data = json.load(json_file)
    c_caffeine_wat = data['analysis'][0]['density']['molecular']['CFF1']['c/M']

# Caffeine in salt solutions
mu_list = [0]
C_list = [0]
for concentration in concentration_list: 
    for tfe in tfe_list:
        if tfe == 0.0178 and (concentration == 0.5 or concentration == 0.4):
            continue
        conc = '{0:.4f}'.format(concentration)
        folder = str(conc)+'_'+str(tfe)+'_'+str((float(conc)*float(tfe)))
        %cd -q $homedir/Simulations/26mM_activity_salts/$folder/production
        with open ('Caffeine_prod.out') as json_file:
            data = json.load(json_file)
            c_caffeine_salt = data['analysis'][0]['density']['molecular']['CFF1']['c/M']
            mu_excess = np.log(c_caffeine_wat/c_caffeine_salt)     # Eq 2
            mu_list.append(mu_excess)
            C_list.append(concentration)

ax.scatter(C_list, mu_list, s=50, color='black')


## BEAUTIFICATION ##
# Sets labels and limits
ax.set_xlabel(r'Salt concentration (mol/l)', weight='bold',  size=16)
ax.set_ylabel(r'Excess Chem. Pot. (kT)', weight='bold', size=16)

# Sets axes ticks
#ax.minorticks_on()
ax.tick_params(which='both', width=2, direction='in')
ax.tick_params(which='major', length=7, direction='in')
ax.tick_params(which='minor', length=3, direction='in')
for tick in list(itertools.chain(*[ax.xaxis.get_major_ticks(), ax.yaxis.get_major_ticks()])):
    tick.label.set_fontsize(14)
    
# Axis limits
ax.set_xlim(0, 0.6)
ax.set_ylim(-0.8, 1)

# Legend
custom_lines = [matplotlib.lines.Line2D([0], [0], color='black', linestyle='None', marker='o', markerfacecolor='none', markersize=8),
                matplotlib.lines.Line2D([0], [0], color='black', linestyle='solid', marker='None', lw=5),
                matplotlib.lines.Line2D([0], [0], color='black', linestyle='None', marker='o', markersize=8)]
ax.legend(custom_lines, ['Exprimental data', 'Experiment fit (linear)', 'Monte Carlo'], fontsize=20, frameon=False)
## END OF BEAUTIFICATION ##

fig.tight_layout()
%cd -q $homedir
fig.savefig('Figures/Chem_pot_exp_v_sim.pdf', bbox_inches='tight')